In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [4]:
matchups = pd.read_csv("data\\matchups\\model_matchups.csv", index_col=0).dropna()

In [5]:
y = matchups['outcome']
x_train = matchups.drop(columns=['outcome','inning','game_code','batter_game_code','pitcher_game_code','batter_id','pitcher_id','b_pos'])

In [6]:
from sklearn import preprocessing
from sklearn.decomposition import PCA

# Change outcome labels to one-hot vectors
le = preprocessing.LabelEncoder()
labels_enc = le.fit_transform(y)
labels = tf.keras.utils.to_categorical(labels_enc)

# Apply standard scaling to features
ss = preprocessing.StandardScaler()
x_standardized = ss.fit_transform(x_train)

# Reduce dimensionality of input data
# pca = PCA(n_components=50)
# principal_components = pca.fit_transform(x_standardized)
# x_standardized = pd.DataFrame(data = principal_components).to_numpy()

In [10]:
p = .05
inputs = tf.keras.layers.Input((73,), name='numeric_inputs')

x = tf.keras.layers.Dropout(p)(inputs)
x = tf.keras.layers.Dense(2500, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(500, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(250, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(100, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
out = tf.keras.layers.Dense(55, activation='softmax', name='output')(x)

model = tf.keras.models.Model(inputs=inputs, outputs=out)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 numeric_inputs (InputLayer)  [(None, 73)]             0         
                                                                 
 dropout_5 (Dropout)         (None, 73)                0         
                                                                 
 dense_4 (Dense)             (None, 100)               7400      
                                                                 
 batch_normalization_4 (Batc  (None, 100)              400       
 hNormalization)                                                 
                                                                 
 dropout_6 (Dropout)         (None, 100)               0         
                                                                 
 output (Dense)              (None, 55)                5555      
                                                           

In [11]:
def bootstrap_sample_generator(batch_size):
    while True:
        batch_idx = np.random.choice(x_standardized.shape[0], batch_size)
        yield ({'numeric_inputs': x_standardized[batch_idx]},
               {'output': labels[batch_idx]})

batch_size = 256

model.fit(
    bootstrap_sample_generator(batch_size),
    steps_per_epoch=20_000 // batch_size,
    epochs=25,
    max_queue_size=20  
)


Epoch 1/25
78/78 [==============================] - 1s 2ms/step - loss: 3.8544 - accuracy: 0.1321
Epoch 2/25
78/78 [==============================] - 0s 2ms/step - loss: 3.0637 - accuracy: 0.2502
Epoch 3/25
78/78 [==============================] - 0s 2ms/step - loss: 2.6637 - accuracy: 0.2782
Epoch 4/25
78/78 [==============================] - 0s 2ms/step - loss: 2.5147 - accuracy: 0.2763
Epoch 5/25
78/78 [==============================] - 0s 2ms/step - loss: 2.4346 - accuracy: 0.2831
Epoch 6/25
78/78 [==============================] - 0s 2ms/step - loss: 2.4059 - accuracy: 0.2847
Epoch 7/25
78/78 [==============================] - 0s 2ms/step - loss: 2.3764 - accuracy: 0.2882
Epoch 8/25
78/78 [==============================] - 0s 2ms/step - loss: 2.3786 - accuracy: 0.2865
Epoch 9/25
78/78 [==============================] - 0s 2ms/step - loss: 2.3419 - accuracy: 0.2943
Epoch 10/25
78/78 [==============================] - 0s 2ms/step - loss: 2.3472 - accuracy: 0.2944
Epoch 11/25
78/78 [